In [1]:
import os
os.chdir('/home/jupyter/chatbot')
res = !gcloud config get core/project
PROJECT_ID = res[0]
import os
print(PROJECT_ID)
os.environ['PROJECT_ID'] = PROJECT_ID
project_id = PROJECT_ID

metal-sky-419309


In [2]:
import os
os.chdir('/home/jupyter/chatbot')

In [3]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
TAG_NAME = 'masterv6'
KAGGLE_USERNAME='andrehpereh1'
# KAGGLE_KEY=''
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-fine-tunning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
BUCKET_NAME = 'able-analyst-416817-chatbot-v1'
# BUCKET_URI = f"gs://{BUCKET_NAME}"
FINE_TUNE_FLAG=  False
EPOCHS=12
MODEL_NAME="gemma_2b_en"


In [ ]:
user = os.getenv("USER_NAME", "")
print(os.path.join(user , "jhdbcjd"))
print(os.path.join("cdsac", user))


In [ ]:
CONTAINER_IMAGE_RUNNING_APP

In [ ]:
substitutions="""
_CONTAINER_IMAGE_NAME_DATA_PREP={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_NAME_DATA_PREP,
           TAG_NAME, 
           ).strip()
print(substitutions)

# Runs the data_preparation component image. (Development, when tested should be moved to the main cloudbuild in the project folder)
# Pay attention to the "." after summit. Might need some changes when move to the master pipeline.
!gcloud builds submit . --timeout=15m --config "components/data_preparation/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}
# DO not forget the tag

In [ ]:
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-data-preparation:latest data_ingestion.py --bucket-name 'able-analyst-416817-chatbot-v1' --directory 'input_data/andrehpereh'

In [ ]:
import vertexai
print(vertexai.__version__)

In [11]:
substitutions="""
_CONTAINER_IMAGE_NAME_FINE_TUNE={},\
_KAGGLE_USERNAME={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_NAME_FINE_TUNE,
           KAGGLE_USERNAME,
           TAG_NAME, 
           ).strip()
print(substitutions)

# Builds image
!gcloud builds submit . --config "components/fine_tunning/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

_CONTAINER_IMAGE_NAME_FINE_TUNE=gemma-chatbot-fine-tunning,_KAGGLE_USERNAME=andrehpereh1,TAG_NAME=masterv6
Creating temporary tarball archive of 55 file(s) totalling 874.6 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/andre/.config/gcloud/logs/2024.04.22/10.42.28.055459.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://able-analyst-416817_cloudbuild/source/1713782548.210487-e674a718116142329b15d4e597bec23e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/able-analyst-416817/locations/us-central1/builds/f16ed7aa-0f2e-4608-83f7-77d49c0de440].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/f16ed7aa-0f2e-4608-83f7-77d49c0de440?project=24796876098 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "f16ed7aa-0f2e-460

In [9]:
substitutions="""
_CONTAINER_IMAGE_RUNNING_APP={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_RUNNING_APP,
           TAG_NAME, 
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/app_flask/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

_CONTAINER_IMAGE_RUNNING_APP=gemma-chatbot-running-app,TAG_NAME=masterv6
Creating temporary tarball archive of 51 file(s) totalling 760.7 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/andre/.config/gcloud/logs/2024.04.23/14.22.50.742527.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://metal-sky-419309_cloudbuild/source/1713882170.89676-5be10540097440f399f9cc75b26af8f4.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/metal-sky-419309/locations/us-central1/builds/a560c4e1-0e0b-4f5d-adcc-03e69d969cc2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/a560c4e1-0e0b-4f5d-adcc-03e69d969cc2?project=450204310329 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "a560c4e1-0e0b-4f5d-adcc-03e69d969cc2"

FETCHSOURCE
Fetchi

In [ ]:
!kfp dsl compile --py './components/pipeline/pipeline/pipeline.py' --output "fine_tune_pipeline_test.json" --function fine_tune_pipeline

In [ ]:
_CONTAINER_IMAGE_RUNNING_APP

In [ ]:
!pwd

In [ ]:
substitutions="""
_CONTAINER_IMAGE_NAME_PIPELINE={},\
TAG_NAME={},\
_BUCKET_NAME={},\
_FINE_TUNE_FLAG={},\
_EPOCHS={},\
_MODEL_NAME={}
""".format(
           CONTAINER_IMAGE_NAME_PIPELINE,
           TAG_NAME,
           BUCKET_NAME,
           FINE_TUNE_FLAG,
           EPOCHS,
           MODEL_NAME
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/pipeline/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
# !python "./components/pipeline/pipeline.py"

In [ ]:
substitutions=f"_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},TAG_NAME={TAG_NAME}"
print(substitutions)
!gcloud builds submit . --config "./components/app_flask/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
PROD_TAG = 'master_v4'
substitutions=f"_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},_PROD_TAG={PROD_TAG}"
!gcloud builds submit . --config "cloudbuil_cloudrun.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [9]:
from cluodbuild_compiler import merge_cloudbuild_files, find_missing_elements
child_files = [ 
    "./components/data_preparation/cloudbuild.yaml", 
    "./components/fine_tunning/cloudbuild.yaml",
    "./components/pipeline/cloudbuild.yaml",
    "./components/app_flask/cloudbuild.yaml"
]

descriptions = [ 
    "Data Preparation",
    "Vertex AI Pipeline Deployment",
    "App flask application (Cloud Run)",
    "Model Training", 
]
all_variables = merge_cloudbuild_files(child_files, descriptions, timeout_hours = 20)

This is all {'steps': [{'name': 'gcr.io/cloud-builders/docker', 'args': ['build', '-t', 'gcr.io/$PROJECT_ID/$_CONTAINER_IMAGE_NAME_DATA_PREP:$TAG_NAME', '.'], 'dir': './components/data_preparation', 'id': 'DP_A', 'waitFor': ['-']}, {'name': 'gcr.io/cloud-builders/docker', 'args': ['push', 'gcr.io/$PROJECT_ID/$_CONTAINER_IMAGE_NAME_DATA_PREP:$TAG_NAME'], 'id': 'DP_B', 'waitFor': ['DP_A']}, {'name': 'gcr.io/cloud-builders/docker', 'args': ['build', '-t', 'gcr.io/$PROJECT_ID/$_CONTAINER_IMAGE_NAME_FINE_TUNE:$TAG_NAME', '--build-arg', 'KAGGLE_KEY=$_KAGGLE_KEY', '--build-arg', 'KAGGLE_USERNAME=$_KAGGLE_USERNAME', '.'], 'dir': './components/fine_tunning', 'id': 'FT_A', 'waitFor': ['-']}, {'name': 'gcr.io/cloud-builders/docker', 'args': ['push', 'gcr.io/$PROJECT_ID/$_CONTAINER_IMAGE_NAME_FINE_TUNE:$TAG_NAME'], 'id': 'FT_B', 'waitFor': ['FT_A']}, {'name': 'gcr.io/cloud-builders/docker', 'args': ['build', '-t', 'gcr.io/$PROJECT_ID/$_CONTAINER_IMAGE_NAME_PIPELINE:$TAG_NAME', '.'], 'dir': './comp

In [10]:
pwd()


'/home/jupyter/chatbot'

In [13]:
substitutions=f"_CONTAINER_IMAGE_NAME_DATA_PREP={CONTAINER_IMAGE_NAME_DATA_PREP},_CONTAINER_IMAGE_NAME_FINE_TUNE={CONTAINER_IMAGE_NAME_FINE_TUNE}\
,_CONTAINER_IMAGE_NAME_PIPELINE={CONTAINER_IMAGE_NAME_PIPELINE},_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},_BUCKET_NAME={BUCKET_NAME}\
_KAGGLE_USERNAME={KAGGLE_USERNAME},_KAGGLE_USERNAME={KAGGLE_USERNAME},_FINE_TUNE_FLAG={FINE_TUNE_FLAG},_EPOCHS={EPOCHS},_MODEL_NAME={MODEL_NAME},TAG_NAME={TAG_NAME}"
print(substitutions)

_CONTAINER_IMAGE_NAME_DATA_PREP=gemma-chatbot-data-preparation,_CONTAINER_IMAGE_NAME_FINE_TUNE=gemma-chatbot-fine-tunning,_CONTAINER_IMAGE_NAME_PIPELINE=gemma-chatbot-pipeline-app,_CONTAINER_IMAGE_RUNNING_APP=gemma-chatbot-running-app,_BUCKET_NAME=able-analyst-416817-chatbot-v1_KAGGLE_USERNAME=andrehpereh1,_KAGGLE_USERNAME=andrehpereh1,_FINE_TUNE_FLAG=False,_EPOCHS=12,_MODEL_NAME=gemma_2b_en,TAG_NAME=masterv6


In [14]:
missing_elements = find_missing_elements(all_variables, substitutions)
print(missing_elements)


[]


In [ ]:
!gcloud builds submit . --config "./master_cloudbuild.json" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
from werkzeug.datastructures import FileStorage

In [ ]:
from google.cloud import bigquery, storage, pubsub_v1

In [ ]:
!gcloud pubsub subscriptions pull

In [ ]:
!gcloud pubsub subscriptions pull --auto-ack your-pipeline-trigger-topic-sub

In [ ]:
import os
from google.cloud import pubsub_v1

# Project and subscription details
project_id = "able-analyst-416817"  
subscription_name = "your-pipeline-trigger-topic-sub"

# Create Subscriber client
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_name)

def callback(message):
    print(f"Received message: {message.data}")
    message.ack()  # Acknowledge the message

# Pulling messages
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

# Wrap subscriber in a 'with' block to automatically close the client when done.
with subscriber:
    try:
        # When `timeout` is notset, result() will block indefinitely,
        # unless an exception is encountered.
        streaming_pull_future.result()  # Continuously pulls messages
    except TimeoutError: 
        streaming_pull_future.cancel()  # Trigger shutdown
        streaming_pull_future.result()  # Block until shutdown is complete 


In [ ]:
! gcloud functions logs read \
  --gen2 \
  --region='us-central1' \
  --limit=5 \
  'function-1'

In [ ]:
!gcloud builds submit . --config "./master_cloudbuild.json" --substitutions {substitutions} --region={GCP_REGION}